# Preamble

First export the tables as an SQL file from phpadmin.

Then fix the SQL file following [these tips](http://livecode.byu.edu/database/mysql-sqliteExport.php).

Finally run this instruction (below) from this [SO answer](https://stackoverflow.com/a/2049137).

This will create an sqlite database that we can connect to.

In [4]:
!cat country_club_db.sql | sqlite3 country_club_db.db

In [5]:
!ls

1584480990_SQLTasks_Tier_2.sql	country_club_db.sql
country_club_db.db		Untitled.ipynb


# Get the engine

In [11]:
import pandas as pd

from sqlalchemy import create_engine

In [8]:
engine = create_engine('sqlite:///country_club_db.db')

In [10]:
engine.table_names()

['Bookings', 'Facilities', 'Members']

# PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

## QUESTIONS:
### Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [44]:
query_q10 = """
WITH total_cost AS (
SELECT
      f.name as facility_name,
      CASE WHEN b.memid = 0 THEN (f.guestcost * b.slots)
      ELSE (f.membercost * b.slots) END AS cost
FROM Bookings AS b
INNER JOIN Facilities as f
  ON b.facid = f.facid
INNER JOIN Members AS m
  ON b.memid = m.memid
)
SELECT facility_name,
       SUM(cost) as total_revenue
FROM total_cost
GROUP BY facility_name
HAVING total_revenue < 1000
ORDER BY total_revenue
"""

In [45]:
rs = pd.read_sql_query(query_q10, engine)

In [46]:
rs

,facility_name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


### Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order

In [64]:
query_q11 = """
SELECT
      m1.firstname AS firstname,
      m1.surname AS surname,
      m2.firstname AS rec_firstname,
      m2.surname AS rec_surname
FROM Members AS m1
LEFT JOIN Members AS m2
  ON m1.recommendedby = m2.memid
ORDER BY m2.surname, m2.firstname
"""

In [65]:
rs = pd.read_sql_query(query_q11, engine)

In [67]:
rs

,firstname,surname,rec_firstname,rec_surname
0,GUEST,GUEST,None,None
1,Darren,Smith,None,None
2,Tracy,Smith,None,None
3,Tim,Rownam,None,None
4,Burton,Tracy,None,None
5,Jemima,Farrell,None,None
6,David,Farrell,None,None
7,Hyacinth,Tupperware,None,None
8,Darren,Smith,None,None
9,Ramnaresh,Sarwin,Florence,Bader


### Q12: Find the facilities with their usage by member, but not guests

In [77]:
query_q12 = """
SELECT
      f.name as facility_name,
      COUNT(b.memid) AS member_usage
FROM Bookings AS b
INNER JOIN Facilities as f
  ON b.facid = f.facid
WHERE b.memid <> 0
GROUP BY facility_name
"""

In [78]:
rs = pd.read_sql_query(query_q12, engine)

In [79]:
rs

,facility_name,member_usage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


### Q13: Find the facilities usage by month, but not guests

In [94]:
query_q13 = """
SELECT
      f.name as facility_name,
      strftime('%m', b.starttime) as month,
      COUNT(b.memid) as monthly_use
FROM Bookings AS b
INNER JOIN Facilities as f
  ON b.facid = f.facid
WHERE b.memid <> 0
GROUP BY facility_name, month
"""

In [95]:
rs = pd.read_sql_query(query_q13, engine)

In [96]:
rs

,facility_name,month,monthly_use
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103
